# A notebook with helpful math functions. 

These will be useful for finding local minimum and saddle points for 

In [58]:
import numpy as np
from scipy.optimize import approx_fprime
import pandas as pd


# Look into getting the gradient / hessian from dft outputs (?)


In [2]:
def gradient ( cost_function, x0, epsilon=1.e-5, linear_approx=False, *args ):
    """
    A numerical approximation to the gradient array of cost_function at
    location x0 (hopefully, the minimum)
    """
    gradient = approx_fprime( x0, cost_function, epsilon) 
    
    if linear_approx:
        gradient = np.matrix(gradient)
        return gradient.transpose() * gradient
    
    return gradient
    

def hessian ( cost_function, x0, epsilon=1.e-5, linear_approx=False, *args ):
    """
    A numerical approximation to the Hessian matrix of cost_function at
    location x0 (hopefully, the minimum)
    
    Code courtsey of https://gist.github.com/jgomezdans
    """
    # ``calculate_cost_function`` is the cost function implementation
    # The next line calculates an approximation to the first
    # derivative
    f1 = approx_fprime( x0, cost_function, epsilon) 

    # This is a linear approximation. Obviously much more efficient
    # if cost function is linear
    if linear_approx:
        f1 = np.matrix(f1)
        return f1.transpose() * f1    
    # Allocate space for the hessian
    n = x0.shape[0]
    hessian = np.zeros ( ( n, n ) )
    # The next loop fill in the matrix
    xx = x0
    for j in xrange( n ):
        xx0 = xx[j] # Store old value
        xx[j] = xx0 + epsilon # Perturb with finite difference
        # Recalculate the partial derivatives for this new point
        f2 = approx_fprime( x0, cost_function, epsilon) 
        hessian[:, j] = (f2 - f1)/epsilon # scale...
        xx[j] = xx0 # Restore initial value of x0        
    return hessian


In [16]:
def calculate_cost_function(z):
    x= z[0]
    y = z[1]
    
    return x**2 + y**3 + x*y

In [17]:
# An input where we know it is a saddle point
inp = np.array([float(0), float(0)])

grad = gradient(calculate_cost_function , inp)
hes = hessian(calculate_cost_function , inp)
print "The Gradient"
print grad
print "The sum of the gradient"
print grad.sum()
print 
print "The Hessian"

print hes
print "The determinant of the Hessian"
print np.linalg.det(hes)

The Gradient
[  1.00000000e-05   1.00000000e-10]
The sum of the gradient
1.00001e-05

The Hessian
[[  2.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   6.00000000e-05]]
The determinant of the Hessian
-0.99988


The above shows that we have found a saddle point. The sum of the gradient was close to 0 and the determinant of the Hessian was negative.

## Now we will add some functionality for CMA-ES... attempting it

In [29]:
X1 = np.random.rand(4)
X2 = np.random.rand(4)
X3 = np.random.rand(4)
X4 = np.random.rand(4)
 


def get_mean_array(pop_list, number_of_best):
    
    count = len(X_list)
    
    
    for X in X_list:
        
    return np.full(len(X), X.mean())

def get_covariance_matrix(X):
    M = get_mean_array(X)
    
    (X-M) * (X-M).T
    
    

In [175]:
def best_function(X):
    return X[0]**2 + X[1]**2
    
    

X1 = np.random.rand(2,1)
X1_score = best_function(X1)
X2 = np.random.rand(2,1)
X2_score = best_function(X2)
X3 = np.random.rand(2,1)
X3_score = best_function(X3)
X4 = np.random.rand(2,1)
X4_score = best_function(X4)

scores = [
    [X1_score, X1],
    [X2_score, X2],
    [X3_score, X3],
    [X4_score, X4]
]
df = pd.DataFrame(scores)
df.columns = ["Score", "Array"]
df.sort("Score")

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:23: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Score                                  Array
0  [0.109448133809]  [[0.322032403272], [0.0757843324963]]
3  [0.126585676162]   [[0.018551456992], [0.355304826319]]
2  [0.550486163083]   [[0.692514021965], [0.266290241025]]
1  [0.829409483363]   [[0.894952957184], [0.168726665917]]

In [176]:
best1, best2 = df.sort("Score").iloc[:2,1]


/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [177]:
best1, best2

(array([[ 0.3220324 ],
        [ 0.07578433]]), array([[ 0.01855146],
        [ 0.35530483]]))

In [178]:
M = (best1 + best2)/2
M

array([[ 0.17029193],
       [ 0.21554458]])

In [179]:
Sigma = (best1-M).dot( (best1-M).T ) + (best2-M).dot( (best2-M).T )
Sigma

array([[ 0.04605034, -0.04241457],
       [-0.04241457,  0.03906585]])

In [180]:
sigma = (best1.std() + best2.std()) / 2
sigma

0.14575036002555206

In [181]:
X3, X4 = M + sigma * np.random.normal(loc=0, scale=1.0, size = (2,2)) * Sigma

In [182]:
X3 = X3.reshape(2,1)
X4 = X4.reshape(2,1)

In [183]:
best1_score = best_function(best1)
best2_score = best_function(best2)
X3_score = best_function(X3)
X4_score = best_function(X4)

In [184]:
scores = [
    [best1_score, best1],
    [best2_score, best2],
    [X3_score, X3],
    [X4_score, X4]
]
df = pd.DataFrame(scores)
df.columns = ["Score", "Array"]
df.sort("Score")

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Score                                  Array
2  [0.0555061139129]   [[0.169166500245], [0.163978075083]]
3   [0.098280991338]   [[0.227514605141], [0.215680541045]]
0   [0.109448133809]  [[0.322032403272], [0.0757843324963]]
1   [0.126585676162]   [[0.018551456992], [0.355304826319]]

In [185]:
df.sort("Score").iloc[0,0]

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


array([ 0.05550611])

In [186]:
X1 = np.random.rand(2,1)
X1_score = best_function(X1)
X2 = np.random.rand(2,1)
X2_score = best_function(X2)
X3 = np.random.rand(2,1)
X3_score = best_function(X3)
X4 = np.random.rand(2,1)
X4_score = best_function(X4)

scores = [
    [X1_score, X1],
    [X2_score, X2],
    [X3_score, X3],
    [X4_score, X4]
]
df = pd.DataFrame(scores)
df.columns = ["Score", "Array"]
df.sort("Score")

print df.sort("Score").iloc[0,0]

for i in range(100) :  
    best1, best2 = df.sort("Score").iloc[:2,1]
    M = (best1 + best2)/2
    Sigma = (best1-M).dot( (best1-M).T ) + (best2-M).dot( (best2-M).T )
    sigma = (best1.std() + best2.std()) / 2
    X3, X4 = M + sigma * np.random.normal(loc=0, scale=1.0, size = (2,2)) * Sigma
    X3 = X3.reshape(2,1)
    X4 = X4.reshape(2,1)
    best1_score = best_function(best1)
    best2_score = best_function(best2)
    X3_score = best_function(X3)
    X4_score = best_function(X4)
    scores = [
        [best1_score, best1],
        [best2_score, best2],
        [X3_score, X3],
        [X4_score, X4]
    ]
    df = pd.DataFrame(scores)
    df.columns = ["Score", "Array"]
    df.sort("Score")
    print df.sort("Score").iloc[0,0]

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:20: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:23: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:42: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:43: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


[ 0.11627209]
[ 0.11627209]
[ 0.06775332]
[ 0.03945148]
[ 0.03945148]
[ 0.03945148]
[ 0.03945148]
[ 0.03945148]
[ 0.03945148]
[ 0.03945148]
[ 0.03937656]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03927941]
[ 0.03

In [187]:
df.sort("Score").iloc[0,1]

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


array([[ 0.14014173],
       [ 0.14014173]])

# Okay, ignoring the CMA-ES stuff for now. We're gonna look at calculating things